In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!nvidia-smi

Wed Mar  6 11:16:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Mistral 7B Instruct



In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
torch.set_default_device('cuda')

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = transformers.BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
model_id,
trust_remote_code=True,
quantization_config=bnb_config,
device_map='auto',
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
model_id,
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
text = "[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen! [INST] Do you have mayonnaise recipes? [/INST]"
encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds

generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen! [INST] Do you have mayonnaise recipes? [/INST]Of course! Mayonnaise is a classic condiment that's easy to make at home. Here's a simple recipe:

Ingredients:
- 1 egg yolk
- 1 tablespoon Dijon mustard
- 1 tablespoon white wine vinegar
- 1/2 teaspoon salt
- 1/4 teaspoon sugar
- 1/2 cup vegetable oil

Instructions:
1. In a small bowl, whisk together the egg yolk, mustard, vinegar, salt, and sugar until well combined.
2. Slowly drizzle in the vegetable oil while whisking continuously to create an emulsion.
3. Continue whisking until the mayonnaise is smooth and creamy.
4. Cover the bowl with a layer of plastic wrap and chill in the refrigerator for at least an hour to allow the flavors to develop.

After ch


### Prompt Format
```
<s>[INST] <user_prompt> [/INST]

<assistant_response> </s>

[INST] <user_prompt>[/INST]
```

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
def generate(input_text, system_prompt="",max_length=512):
    prompt = f"""<s>[INST]{input_text}[/INST]"""
    inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False)
    model_inputs = encodeds.to(device)
    #model.to(device)
    outputs = model.generate(**inputs,
                             max_length=max_length,
                             temperature=0.1,
                             do_sample=True)
    text = tokenizer.batch_decode(outputs)[0]
    wrapped_text = wrap_text(text)
    print(wrapped_text)

## Chat

In [ ]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion?

 Bob: I've found that taking short breaks and doing something different for a few minutes
can help. Maybe go for a walk, listen to music or do a quick meditation.

 Alice: That's a good idea, I'll give it a try.

 Bob: Also, make sure you're getting enough sleep and eating well. A healthy body and mind
can help with focus and productivity.

 Alice: I'll make sure to prioritize those things too.




In [ ]:
generate("What are some coping mechanisms for dealing with depression?",
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What are some coping mechanisms for dealing with depression?

1. Seek professional help: Talk to a therapist or counselor who can help you understand
your feelings and provide guidance on how to manage them.

2. Engage in physical activity: Regular exercise has been shown to improve mood and reduce
symptoms of depression.

3. Practice mindfulness: Mindfulness techniques such as meditation and deep breathing can
help you stay present and focused, reducing stress and anxiety.

4. Connect with others: Spend time with friends and family members who support you and
make you feel valued.

5. Get enough sleep: Adequate sleep is essential for overall well-being and can help
alleviate symptoms of depression.

6. Eat a healthy diet: A balanced diet rich in fruits, vegetables, whole grains, and lean
protein can improve your physical health and boost your mood.

7. Set realistic goals: Break down large tasks into smaller, manageable steps to avoid
feeling overwhelmed.

8. Learn to say no: Don't ta

In [ ]:
generate("What are some self-care practices that can help with depression?",
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What are some self-care practices that can help with depression?

1. Exercise regularly: Physical activity can help release endorphins, which are natural
mood-boosters. Aim for at least 30 minutes of moderate exercise most days of the week.

2. Get enough sleep: Lack of sleep can exacerbate symptoms of depression. Try to establish
a consistent sleep schedule by going to bed and waking up at the same time each day.

3. Eat a healthy diet: A balanced diet rich in fruits, vegetables, whole grains, and lean
protein can help improve overall health and mood.

4. Practice mindfulness: Mindfulness techniques such as meditation, deep breathing, or
yoga can help reduce stress and increase self-awareness.

5. Connect with others: Social support from friends, family, or a support group can
provide comfort and encouragement during difficult times.

6. Engage in enjoyable activities: Doing things you enjoy, whether it's reading, listening
to music, or spending time outdoors, can help boost mood and 

In [ ]:
generate("Can you suggest some relaxation techniques for managing depression?",
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Can you suggest some relaxation techniques for managing depression?

1. Mindfulness meditation: This technique involves focusing on the present moment and
accepting it without judgment. It can help reduce stress and anxiety, and improve mood.
2. Deep breathing exercises: Slow, deep breaths can help activate the body's relaxation
response and reduce feelings of anxiety and stress.
3. Progressive muscle relaxation: This technique involves tensing and relaxing different
muscle groups in the body to release tension and promote relaxation.
4. Visualization: Visualizing a peaceful scene or place can help reduce stress and anxiety
and improve mood.
5. Yoga or tai chi: These practices combine physical movements with breathing and
meditation techniques to promote relaxation and reduce stress.
6. Exercise: Regular physical activity can help improve mood, reduce stress, and promote
better sleep.
7. Social support: Spending time with friends and family members can help reduce feelings
of isolation

In [ ]:
generate("How can I talk to my loved ones about my struggles with depression?",
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


How can I talk to my loved ones about my struggles with depression?

It can be difficult to talk about your struggles with depression, but it's important to
remember that you are not alone. Here are some tips to help you start the conversation
with your loved ones:

1. Choose a time and place that feels comfortable for you. You may want to talk to someone
in person, over the phone, or through video chat.
2. Start by sharing how you're feeling. It can be helpful to use "I" statements to express
your thoughts and feelings without blaming or accusing anyone. For example, "I've been
feeling really down lately" or "I've been struggling with anxiety and it's been affecting
my daily life."
3. Be honest about what you're going through. It's okay to share the details of your
struggles, but try to keep it in a way that doesn't make you feel like a burden.
4. Listen to your loved ones' responses. They may have questions or concerns, and it's
important to give them a chance to express themselves.


In [ ]:
generate("Are there any online support groups for people with depression?",
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Are there any online support groups for people with depression?
A: Yes, there are many online support groups for people with depression. Some popular
options include Reddit's r/depression community, the Depression and Bipolar Support
Alliance (DBSA), and the National Alliance on Mental Illness (NAMI). Additionally, there
are many online therapy platforms that offer virtual support groups and individual therapy
sessions.</s>


In [ ]:
generate("What are the signs that someone might be experiencing depression?",
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What are the signs that someone might be experiencing depression?

1. Persistent feelings of sadness, hopelessness, or emptiness.
2. Loss of interest or pleasure in activities that were once enjoyable.
3. Changes in appetite and weight.
4. Difficulty sleeping or sleeping too much.
5. Fatigue or loss of energy.
6. Feelings of worthlessness, guilt, or helplessness.
7. Difficulty concentrating or making decisions.
8. Thoughts of death or suicide.

It's important to note that everyone experiences these symptoms differently, and not
everyone with depression will exhibit all of them. If you're concerned about yourself or
someone else, it's best to speak with a mental health professional.</s>


In [ ]:
generate("Can you provide tips for maintaining a healthy lifestyle while dealing with depression?",
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Can you provide tips for maintaining a healthy lifestyle while dealing with depression?

1. Exercise regularly: Exercise is a great way to boost your mood and reduce stress. Aim
for at least 30 minutes of moderate exercise most days of the week.

2. Get enough sleep: Lack of sleep can exacerbate depression symptoms. Try to establish a
regular sleep schedule and stick to it as much as possible.

3. Eat a balanced diet: Eating a healthy, balanced diet can help improve your mood and
energy levels. Avoid processed foods and focus on whole grains, fruits, vegetables, lean
protein, and healthy fats.

4. Stay connected with others: Social support is crucial for maintaining mental health.
Try to spend time with friends and family members, join a support group, or seek
professional help if needed.

5. Practice mindfulness: Mindfulness techniques such as meditation, deep breathing, and
yoga can help reduce stress and improve mood.

6. Avoid alcohol and drugs: Alcohol and drugs can worsen depress

In [ ]:
generate("How can I build resilience and bounce back from depressive episodes?",
         max_length=256)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


How can I build resilience and bounce back from depressive episodes?

## Answer (1)

I'm not a psychologist, but I've been through a lot of depression and anxiety.

The first thing you need to do is to stop thinking about it. You can't control your
thoughts, but you can control what you focus on.

If you're thinking about how bad you feel, you're not thinking about anything else. You're
not thinking about what you can do to improve your situation. You're not thinking about
what you can do to help yourself.

So, stop thinking about it.

Instead, focus on what you can do to improve your situation.

What can you do to make yourself feel better?

What can you do to help yourself?

What can you do to make your life better?

You can't control your thoughts, but you can control what you focus on.

So, focus on what you can do to improve your situation.

If you're feeling bad, don't think about it. Instead, focus on what you can do to improve
your situation.

If you


In [ ]:
generate("Are there any natural remedies or supplements that can help alleviate depression symptoms?",
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Are there any natural remedies or supplements that can help alleviate depression symptoms?

Yes, there are several natural remedies and supplements that may help alleviate depression
symptoms. Here are a few:

1. St. John's Wort: St. John's Wort is an herb that has been used for centuries to treat
depression. It works by increasing the levels of serotonin in the brain, which can help
improve mood.
2. Omega-3 Fatty Acids: Omega-3 fatty acids are found in fish oil and have been shown to
help reduce symptoms of depression. They work by reducing inflammation in the brain and
improving brain function.
3. Saffron: Saffron is a spice that has been used for centuries to treat depression. It
works by increasing the levels of serotonin and dopamine in the brain, which can help
improve mood.
4. L-Theanine: L-Theanine is an amino acid that is found in tea. It works by reducing
stress and anxiety, which can help improve mood.
5. Valerian Root: Valerian root is an herb that has been
